# 提取特征值

In [1]:
import ijson
import pandas as pd
from tqdm import tqdm
import csv
import os

In [2]:
input_file = "json_data.json"
output_file = "dataset.csv"

# 用于存储每个流的信息
flows = {}

In [3]:
# 获取文件的总行数
def get_file_line_count(file_path):
    with open(file_path, "r") as f:
        return sum(1 for line in f)

In [4]:
# total_lines = get_file_line_count(input_file)

# 解析 JSON
with open(input_file, "r", encoding='utf-8') as f:
    objects = ijson.items(f, '')
    print(objects.__next__())
    # for packet in objects:
    #     # for packet in tqdm(objects, total=total_lines, desc="Processing"):
    #     print(packet)
    #     if "layers" not in packet:
    #         continue

    #     layers = packet["layers"]
    #     if "tcp" not in layers:
    #         continue

    #     stream_index = layers["tcp"]["stream"]
    #     payload_length = len(layers["tcp"].get("payload", ""))
    #     sni = None

    #     if "tls" in layers:
    #         tls_layer = layers["tls"]
    #         if (
	# 			"handshake" in tls_layer
	# 			and tls_layer["handshake"]["type"] == "client_hello"
	# 		):
    #             sni = tls_layer["handshake"].get("sni", None)

    #     if stream_index not in flows:
    #         flows[stream_index] = {"sni": sni, "lengths": []}

    #     if payload_length > 0:
    #         flows[stream_index]["lengths"].append(payload_length)

In [ ]:
# 生成最终的 CSV 文件数据
data = []
for stream_index, flow in flows.items():
	sni = flow["sni"]
	lengths = flow["lengths"][:30] + [0] * (30 - len(flow["lengths"]))
	data.append([input_file, stream_index, sni] + lengths)

# 创建 DataFrame 并保存为 CSV
df = pd.DataFrame(
	data,
	columns=["filename", "stream_index", "sni"] + [f"len{i}" for i in range(1, 31)],
)
df.to_csv(output_file, index=False)

print(f"Dataset saved to {output_file}")

Dataset saved to dataset.csv
